In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from diffusers import FlaxStableDiffusionXLPipeline
pipe, params = FlaxStableDiffusionXLPipeline.from_pretrained(
    "nev/lcm-sdxl-flax"
)

/home/neverix/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 19 files: 100%|██████████| 19/19 [00:00<00:00, 76553.10it/s]
The config attributes {'force_zeros_for_empty_prompt': True, 'add_watermarker': None} were passed to FlaxStableDiffusionXLPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'force_zeros_for_empty_prompt': True, 'add_watermarker': None} are not expected by FlaxStableDiffusionXLPipeline and will be ignored.
The config attributes {'interpolation_type': 'linear', 'use_karras_sigmas': False} were passed to FlaxEulerDiscreteScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
The config attributes {'act_fn': 'silu', 'attention_type': 'default', 'center_input_sample': False, 'class_embed_type': None, 'class_embeddings_concat': False, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'cross_attention_norm': None, 'downsample_padding': 1, 'dual_cross_attention': False, 'encoder_hid_dim': None, 'enc

/home/neverix/.cache/huggingface/hub/models--nev--lcm-sdxl-flax/snapshots/52fb47f31f449167eb7ab744a2ff35e006ff8ec8/unet


The config attributes {'force_upcast': True} were passed to FlaxAutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


/home/neverix/.cache/huggingface/hub/models--nev--lcm-sdxl-flax/snapshots/52fb47f31f449167eb7ab744a2ff35e006ff8ec8/vae


In [3]:
import jax
print(jax.devices())

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


In [4]:
import jax.numpy as jnp


dtype = jnp.float32
params = jax.tree_util.tree_map(lambda x: x.astype(dtype), params)

In [5]:
from flax.training.common_utils import shard
from flax.jax_utils import replicate
import numpy as np


p_params = replicate(params)

def create_key(seed=0):
    return jax.random.PRNGKey(seed)

rng = create_key(0)
rng = jax.random.split(rng, jax.device_count())

do_jit = True

def generate(prompt_ids):
    images = pipe(
        prompt_ids if do_jit else prompt_ids[0],
        p_params if do_jit else params,
        rng if do_jit else rng[0],
        num_inference_steps=1,
        # neg_prompt_ids=neg_prompt_ids if do_jit else neg_prompt_ids[0],
        guidance_scale = 1.0,
        jit=do_jit,
    ).images
    images = images.reshape((images.shape[0] * images.shape[1], ) + images.shape[-3:])
    return pipe.numpy_to_pil(np.array(images))

In [6]:
imgs_per_device = 1

prompt = "a cat"
prompts = [prompt] * jax.device_count() * imgs_per_device
prompt_ids = pipe.prepare_inputs(prompts)

prompt_ids = shard(prompt_ids)

for u in generate(prompt_ids):
    display(u)

XlaRuntimeError: RESOURCE_EXHAUSTED: XLA:TPU compile permanent error. Ran out of memory in memory space hbm. Used 15.54G of 15.48G hbm. Exceeded hbm capacity by 57.28M.

Total hbm usage >= 16.06G:
    reserved        530.00M 
    program           2.61G 
    arguments        12.93G 

Output size 12.00M; shares 0B with arguments.

Program hbm requirement 2.61G:
    global            83.0K
    scoped           161.0K
    HLO temp          2.54G (100.0% utilization: Unpadded (2.03G) Padded (2.03G), 20.1% fragmentation (522.18M))
    overlays         65.82M

  Largest program allocations in hbm:

  1. Size: 524.00M
     Shape: bf16[1024,8,131,256]{3,0,2,1:T(8,128)(2,1)}
     Unpadded size: 524.00M
     XLA label: fusion.85.remat.1 = fusion(slice.509, bitcast.2878, bitcast.7047), kind=kLoop, calls=fused_computation.73.clone.1
     Allocation type: HLO temp
     ==========================

  2. Size: 524.00M
     Shape: bf16[1024,8,131,256]{3,0,2,1:T(8,128)(2,1)}
     Unpadded size: 524.00M
